# Démonstration avancée Qwen3-TTS 1.7B
## Conception de voix, contrôle par instructions et clonage vocal

Ce notebook présente les fonctionnalités avancées des modèles Qwen3-TTS 1.7B.

**GPU requis** : Exécution > Modifier le type d'exécution > GPU (T4 ou supérieur)

## 1 - Installation et configuration

In [ ]:
!pip install -U qwen-tts soundfile -q
!pip install flash-attn --no-build-isolation -q

In [ ]:
import torch
import soundfile as sf
import os
from IPython.display import Audio, display, Markdown
print(f"PyTorch : {torch.__version__}")
print(f"CUDA disponible : {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU : {torch.cuda.get_device_name(0)}")
os.makedirs("audio_outputs", exist_ok=True)
try:
    import flash_attn
    ATTN_IMPL = "flash_attention_2"
    print("Flash Attention 2 disponible")
except ImportError:
    ATTN_IMPL = "eager"
    print("Flash Attention non disponible")

In [ ]:
def play_audio(wav, sr, filename=None, title=None):
    if filename:
        sf.write(f"audio_outputs/{filename}.wav", wav, sr)
    if title:
        display(Markdown(f"**{title}**"))
    display(Audio(wav, rate=sr))

## PARTIE 1 : Conception de voix (VoiceDesign)

In [ ]:
from qwen_tts import Qwen3TTSModel
torch.cuda.empty_cache()
print("Chargement du modèle VoiceDesign...")
voice_design_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modèle VoiceDesign chargé !")

In [ ]:
print("Génération : Voix de fille animé mignonne")
wavs, sr = voice_design_model.generate_voice_design(
    text="Coucou tout le monde ! Je suis tellement contente de vous rencontrer aujourd'hui ! On va bien s'amuser ensemble !",
    language="French",
    instruct="Voix féminine jeune, environ 16 ans, très mignonne et énergique. Aiguë avec un ton vif et joyeux. Parle avec enthousiasme, comme un personnage d'animé. Petites exclamations adorables."
)
play_audio(wavs[0], sr, "vd_01_fille_anime_fr", "Voix de fille animé mignonne")

In [ ]:
print("Génération : Voix de narrateur de documentaire")
wavs, sr = voice_design_model.generate_voice_design(
    text="Dans les profondeurs de l'océan, là où la lumière du soleil ne pénètre jamais, se cache un monde peuplé de créatures extraordinaires.",
    language="French",
    instruct="Voix masculine grave, 50-60 ans, baryton profond. Style narrateur documentaire français. Parle lentement avec gravité et autorité. Pauses naturelles pour créer du suspense."
)
play_audio(wavs[0], sr, "vd_02_narrateur_fr", "Narrateur de documentaire")

In [ ]:
print("Génération : Voix d'adolescent nerveux")
wavs, sr = voice_design_model.generate_voice_design(
    text="Euh, s-salut... Je me demandais si peut-être... tu vois... si t'es pas trop occupée... on pourrait réviser ensemble ?",
    language="French",
    instruct="Voix masculine jeune, 17 ans, ténor. Nerveux et timide, voix légèrement tremblante. Hésitations et pauses, trébuchant sur les mots avec bégaiements occasionnels."
)
play_audio(wavs[0], sr, "vd_03_ado_nerveux_fr", "Adolescent nerveux")

In [ ]:
print("Génération : Voix de méchant mystérieux")
wavs, sr = voice_design_model.generate_voice_design(
    text="Tu pensais pouvoir t'échapper ? Comme c'est délicieusement naïf. Le jeu ne fait que commencer.",
    language="French",
    instruct="Voix masculine menaçante, âge moyen, grave et soyeuse. Parle lentement avec sous-entendus sinistres. Confiant et moqueur."
)
play_audio(wavs[0], sr, "vd_04_mechant_fr", "Méchant mystérieux")

In [ ]:
test_line = "Je n'arrive pas à croire que ça se passe vraiment en ce moment."
voice_designs = [
    ("Enfant excitée", "Fille de 8 ans, voix très aiguë et excitée. Parle vite avec enthousiasme."),
    ("Adulte épuisée", "Femme de 35 ans, fatiguée et exaspérée. Soupire, parle lentement."),
    ("Ancien choqué", "Homme de 70 ans, voix rauque, choqué et incrédule."),
    ("Ado sarcastique", "Fille de 16 ans, sarcasme qui dégouline. Monotone.")
]
print(f"Texte : {test_line}")
for i, (name, instruct) in enumerate(voice_designs):
    print(f"{name}")
    wavs, sr = voice_design_model.generate_voice_design(text=test_line, language="French", instruct=instruct)
    play_audio(wavs[0], sr, f"vd_05_meme_texte_fr_{i+1}")

In [ ]:
# Voix françaises personnalisées - Personnages typiques
print("🇫🇷 VOIX FRANÇAISES PERSONNALISÉES\n")

voix_francaises = [
    ("Bonjour à tous, bienvenue dans mon cours de philosophie. Aujourd'hui, nous allons explorer la pensée de Descartes.",
     "Professeur universitaire français, 55 ans, voix posée et cultivée. Style Sorbonne, articulation précise.",
     "Professeur"),
    ("Bonjour ! Qu'est-ce que je vous sers aujourd'hui ? Les croissants sont tout frais, ils sortent du four !",
     "Boulangère parisienne, 45 ans, chaleureuse et dynamique. Accent francilien, accueillante.",
     "Boulangère"),
    ("Suivez-moi, je vais vous faire découvrir les secrets de ce magnifique château de la Loire.",
     "Guide touristique passionnée, 35 ans, cultivée et enthousiaste. Voix claire et engageante.",
     "Guide"),
    ("Mesdames et messieurs, votre attention s'il vous plaît. Le train à destination de Marseille va entrer en gare.",
     "Annonceur SNCF, voix masculine professionnelle, claire et neutre. Diction parfaite.",
     "Annonceur"),
]

for text, instruct, title in voix_francaises:
    print(f"--- {title} ---")
    wavs, sr = voice_design_model.generate_voice_design(text=text, language="French", instruct=instruct)
    play_audio(wavs[0], sr, f"vd_06_voix_fr_{title.lower()}")

## PARTIE 2 : Contrôle par instructions (CustomVoice)

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modèle CustomVoice 1.7B...")
custom_voice_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-CustomVoice", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modèle CustomVoice 1.7B chargé !")
print(f"Locuteurs disponibles : {', '.join(custom_voice_model.get_supported_speakers())}")

In [ ]:
emotion_text = "Je viens d'apprendre ce qui s'est passé hier."
speaker = "Serena"
emotions = [
    ("Joyeuse", "Ton très joyeux et excité"),
    ("Triste", "Triste et mélancolique, voix légèrement brisée"),
    ("En colère", "En colère et frustrée, parlant avec force"),
    ("Apeurée", "Effrayée et anxieuse, voix tremblante"),
    ("Neutre", "")
]
print(f"Texte : {emotion_text}")
for emotion_name, instruct in emotions:
    print(f"{emotion_name}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=emotion_text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_01_emotion_fr_{emotion_name.lower().replace(' ', '_')}")

In [ ]:
style_text = "S'il vous plaît, faites silence, le bébé dort."
speaker = "Serena"
styles = [
    ("Chuchotement", "Chuchotant très doucement et silencieusement"),
    ("Fort", "Parlant fort et clairement, projetant la voix"),
    ("Rapide", "Parlant très vite, rythme pressé"),
    ("Lent", "Parlant très lentement et délibérément"),
    ("Dramatique", "Très dramatique et théâtral")
]
print(f"Texte : {style_text}")
for style_name, instruct in styles:
    print(f"{style_name}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=style_text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_02_style_fr_{style_name.lower()}")

In [ ]:
roleplay_scenarios = [
    ("Uncle_Fu", "Bienvenue à ce match de championnat ! La tension ici est absolument électrique !", "Commentateur sportif, très énergique et excité", "Commentateur"),
    ("Serena", "Notre chiffre d'affaires trimestriel a augmenté de quinze pour cent.", "Présentation professionnelle, confiante et claire", "Presentatrice"),
    ("Uncle_Fu", "Et alors le dragon s'est retourné vers notre groupe... lancez les dés !", "Maître du donjon, mystérieux et dramatique", "Donjon"),
    ("Serena", "La recette du jour, c'est une délicieuse omelette au riz !", "Animatrice cuisine joyeuse et chaleureuse", "Cuisine")
]
print("SCÉNARIOS DE JEU DE RÔLE")
for speaker, text, instruct, title in roleplay_scenarios:
    print(f"{title} - {speaker}")
    wavs, sr = custom_voice_model.generate_custom_voice(text=text, language="French", speaker=speaker, instruct=instruct)
    play_audio(wavs[0], sr, f"cv_03_scenario_fr_{title.lower()}")

## PARTIE 3 : Clonage vocal (Base)

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modèle Base (clonage vocal)...")
clone_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-Base", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modèle Base chargé !")

In [ ]:
# IMPORTANT : Remplacez par votre propre fichier audio
ref_audio_url = "/content/votre_audio_reference.mp3"
ref_text = "Ceci est la transcription exacte de ce qui est dit dans votre audio de référence."
print(f"Audio de référence : {ref_audio_url}")
new_texts = [
    "Bonjour, ceci est un test de clonage vocal en français.",
    "Le temps aujourd'hui est magnifique, il fait beau et chaud.",
    "Cette technologie de clonage vocal est vraiment impressionnante."
]
for i, text in enumerate(new_texts):
    print(f"Génération : {text}")
    wavs, sr = clone_model.generate_voice_clone(text=text, language="French", ref_audio=ref_audio_url, ref_text=ref_text)
    play_audio(wavs[0], sr, f"clone_01_echantillon_fr_{i+1}")

## PARTIE 4 : Conception puis Clonage

In [ ]:
torch.cuda.empty_cache()
print("Chargement du modèle VoiceDesign...")
design_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-VoiceDesign", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
print("Modèle VoiceDesign chargé !")

In [ ]:
print("ÉTAPE 1 : Concevoir la voix du personnage")
character_instruct = "Homme, âge environ 70 ans, voix grave et rocailleuse avec sagesse. Parle lentement et pensivement. Chaleureux mais autoritaire."
reference_text = "Ah, jeune apprenti, tu cherches la connaissance des arts anciens. Très bien, je vais t'enseigner les secrets de la magie."
ref_wavs, sr = design_model.generate_voice_design(text=reference_text, language="French", instruct=character_instruct)
sf.write("audio_outputs/sorcier_reference.wav", ref_wavs[0], sr)
print("Audio de référence généré !")
play_audio(ref_wavs[0], sr, title="Voix du vieux sorcier")

In [ ]:
torch.cuda.empty_cache()
print("ÉTAPE 2 : Créer le clone vocal du sorcier")
clone_model = Qwen3TTSModel.from_pretrained("Qwen/Qwen3-TTS-12Hz-1.7B-Base", device_map="cuda:0", dtype=torch.bfloat16, attn_implementation=ATTN_IMPL)
sorcier_prompt = clone_model.create_voice_clone_prompt(ref_audio=(ref_wavs[0], sr), ref_text=reference_text)
print("Prompt vocal du sorcier créé avec succès !")

In [ ]:
print("ÉTAPE 3 : Générer les dialogues du sorcier")
dialogues_sorcier = [
    "Le chemin de la magie est long et périlleux, mon enfant.",
    "En sept cents ans de vie, j'ai appris que la patience est la plus grande des vertus.",
    "Méfie-toi de la forêt sombre qui se trouve à l'est du royaume.",
    "Tu as bien travaillé aujourd'hui, jeune apprenti. Je suis fier de toi.",
    "Maintenant, commençons ton entraînement aux arts mystiques."
]
for i, replique in enumerate(dialogues_sorcier):
    print(f"Réplique {i+1} : {replique}")
    wavs, sr = clone_model.generate_voice_clone(text=replique, language="French", voice_clone_prompt=sorcier_prompt)
    play_audio(wavs[0], sr, f"sorcier_replique_fr_{i+1}")

## Téléchargement des fichiers

In [ ]:
import shutil
from google.colab import files
shutil.make_archive("qwen3_tts_outputs_fr", "zip", "audio_outputs")
print("Fichiers compressés !")
for f in sorted(os.listdir("audio_outputs")):
    if f.endswith(".wav"):
        print(f"  {f}")
files.download("qwen3_tts_outputs_fr.zip")